## 1. Setup and Imports

In [ ]:
# Data handling
import pandas as pd
import numpy as np
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Machine Learning Models
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

# PyTorch for Neural Network
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

# Set random seeds for reproducibility
np.random.seed(42)
torch.manual_seed(42)

# Display settings
pd.set_option('display.max_columns', None)
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)

# Check PyTorch device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print("✅ All libraries imported successfully!")
print(f"PyTorch version: {torch.__version__}")
print(f"Device: {device}")
print(f"Pandas version: {pd.__version__}")
print(f"NumPy version: {np.__version__}")

## 2. Load Merged Solar Power Data

In [ ]:
# Load the merged datasets
data_dir = Path('../datasets/solar/processed')

plant1 = pd.read_csv(data_dir / 'plant1_merged.csv', parse_dates=['DATE_TIME'])
plant2 = pd.read_csv(data_dir / 'plant2_merged.csv', parse_dates=['DATE_TIME'])

print("Plant 1 Data:")
print(f"  Shape: {plant1.shape}")
print(f"  Columns: {list(plant1.columns)}")
print(f"\nPlant 2 Data:")
print(f"  Shape: {plant2.shape}")
print(f"  Columns: {list(plant2.columns)}")

# Display first few rows
print("\n" + "="*80)
print("PLANT 1 - First 5 Rows:")
print("="*80)
display(plant1.head())

print("\n" + "="*80)
print("Data Statistics:")
print("="*80)
display(plant1.describe())

## 3. Data Preprocessing and Feature Engineering

In [ ]:
def preprocess_solar_data(df):
    """
    Preprocess solar power data:
    - Extract time-based features
    - Handle missing values
    - Create additional features
    """
    df = df.copy()
    
    # Extract time-based features
    df['HOUR'] = df['DATE_TIME'].dt.hour
    df['DAY'] = df['DATE_TIME'].dt.day
    df['MONTH'] = df['DATE_TIME'].dt.month
    df['DAY_OF_WEEK'] = df['DATE_TIME'].dt.dayofweek
    df['DAY_OF_YEAR'] = df['DATE_TIME'].dt.dayofyear
    
    # Cyclical encoding for hour and month
    df['HOUR_SIN'] = np.sin(2 * np.pi * df['HOUR'] / 24)
    df['HOUR_COS'] = np.cos(2 * np.pi * df['HOUR'] / 24)
    df['MONTH_SIN'] = np.sin(2 * np.pi * df['MONTH'] / 12)
    df['MONTH_COS'] = np.cos(2 * np.pi * df['MONTH'] / 12)
    
    # Create interaction features
    if 'AMBIENT_TEMPERATURE' in df.columns and 'IRRADIATION' in df.columns:
        df['TEMP_IRRAD_INTERACTION'] = df['AMBIENT_TEMPERATURE'] * df['IRRADIATION']
    
    if 'MODULE_TEMPERATURE' in df.columns and 'IRRADIATION' in df.columns:
        df['MODULE_TEMP_IRRAD'] = df['MODULE_TEMPERATURE'] * df['IRRADIATION']
    
    # Is daytime flag (hour between 6 AM and 6 PM)
    df['IS_DAYTIME'] = ((df['HOUR'] >= 6) & (df['HOUR'] <= 18)).astype(int)
    
    # Handle missing values
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].median())
    
    return df

# Preprocess Plant 1
plant1_processed = preprocess_solar_data(plant1)

print("✅ Data preprocessing completed!")
print(f"\nPlant 1 - New shape: {plant1_processed.shape}")
print(f"\nNew features added: {[col for col in plant1_processed.columns if col not in plant1.columns]}")

## 4. Prepare Training and Testing Data

In [ ]:
def prepare_ml_data(df, target_col='DC_POWER'):
    """
    Prepare data for machine learning models
    """
    # Select features
    exclude_cols = ['DATE_TIME', 'SOURCE_KEY', 'PLANT_ID', target_col]
    if 'AC_POWER' in df.columns and target_col != 'AC_POWER':
        exclude_cols.append('AC_POWER')
    
    feature_cols = [col for col in df.columns if col not in exclude_cols]
    
    X = df[feature_cols].copy()
    y = df[target_col].copy()
    
    # Handle any remaining NaN values
    X = X.fillna(X.median())
    y = y.fillna(y.median())
    
    # Split data (80% train, 20% test)
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, shuffle=False  # shuffle=False to preserve time order
    )
    
    # Scale features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Convert back to DataFrame for easier handling
    X_train_scaled = pd.DataFrame(X_train_scaled, columns=feature_cols, index=X_train.index)
    X_test_scaled = pd.DataFrame(X_test_scaled, columns=feature_cols, index=X_test.index)
    
    return X_train_scaled, X_test_scaled, y_train, y_test, scaler, feature_cols

# Prepare data for Plant 1
X_train, X_test, y_train, y_test, scaler, features = prepare_ml_data(plant1_processed)

print("="*80)
print("DATA PREPARATION COMPLETE")
print("="*80)
print(f"\nTraining set: {X_train.shape}")
print(f"Testing set: {X_test.shape}")
print(f"Number of features: {len(features)}")
print(f"\nFeatures used:\n{features}")
print(f"\nTarget variable: DC_POWER")
print(f"  Training range: [{y_train.min():.2f}, {y_train.max():.2f}]")
print(f"  Testing range: [{y_test.min():.2f}, {y_test.max():.2f}]")

## 5. Model 1: Linear Regression

**Theory:** Finds the best-fitting straight line through the data

**Equation:** $\hat{y} = \beta_0 + \beta_1x_1 + \beta_2x_2 + ... + \beta_nx_n$

**Method:** Ordinary Least Squares (OLS) - minimizes sum of squared residuals

In [ ]:
print("Training Linear Regression Model...")
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)

# Predictions
y_train_pred_lr = lr_model.predict(X_train)
y_test_pred_lr = lr_model.predict(X_test)

# Evaluate
lr_train_mse = mean_squared_error(y_train, y_train_pred_lr)
lr_test_mse = mean_squared_error(y_test, y_test_pred_lr)
lr_train_rmse = np.sqrt(lr_train_mse)
lr_test_rmse = np.sqrt(lr_test_mse)
lr_train_mae = mean_absolute_error(y_train, y_train_pred_lr)
lr_test_mae = mean_absolute_error(y_test, y_test_pred_lr)
lr_train_r2 = r2_score(y_train, y_train_pred_lr)
lr_test_r2 = r2_score(y_test, y_test_pred_lr)

print("\n" + "="*70)
print("LINEAR REGRESSION RESULTS")
print("="*70)
print(f"\n📊 Training Set:")
print(f"   MSE:  {lr_train_mse:.4f}")
print(f"   RMSE: {lr_train_rmse:.4f}")
print(f"   MAE:  {lr_train_mae:.4f}")
print(f"   R²:   {lr_train_r2:.4f}")

print(f"\n📊 Testing Set:")
print(f"   MSE:  {lr_test_mse:.4f}")
print(f"   RMSE: {lr_test_rmse:.4f}")
print(f"   MAE:  {lr_test_mae:.4f}")
print(f"   R²:   {lr_test_r2:.4f}")
print("="*70)

## 6. Model 2: Polynomial Regression (Degree 2)

**Theory:** Extends linear regression by adding polynomial terms to capture non-linear relationships

**Equation:** $\hat{y} = \beta_0 + \beta_1x_1 + \beta_2x_1^2 + \beta_3x_2 + \beta_4x_2^2 + ...$

**Method:** Create polynomial features, then apply linear regression

In [ ]:
print("Training Polynomial Regression Model (Degree 2)...")

# Create polynomial features (degree 2)
poly = PolynomialFeatures(degree=2, include_bias=False)
X_train_poly = poly.fit_transform(X_train)
X_test_poly = poly.transform(X_test)

print(f"\n📊 Polynomial features created: {X_train_poly.shape[1]} features (from {X_train.shape[1]} original)")

# Train polynomial regression
poly_model = LinearRegression()
poly_model.fit(X_train_poly, y_train)

# Predictions
y_train_pred_poly = poly_model.predict(X_train_poly)
y_test_pred_poly = poly_model.predict(X_test_poly)

# Evaluate
poly_train_mse = mean_squared_error(y_train, y_train_pred_poly)
poly_test_mse = mean_squared_error(y_test, y_test_pred_poly)
poly_train_rmse = np.sqrt(poly_train_mse)
poly_test_rmse = np.sqrt(poly_test_mse)
poly_train_mae = mean_absolute_error(y_train, y_train_pred_poly)
poly_test_mae = mean_absolute_error(y_test, y_test_pred_poly)
poly_train_r2 = r2_score(y_train, y_train_pred_poly)
poly_test_r2 = r2_score(y_test, y_test_pred_poly)

print("\n" + "="*70)
print("POLYNOMIAL REGRESSION RESULTS")
print("="*70)
print(f"\n📊 Training Set:")
print(f"   MSE:  {poly_train_mse:.4f}")
print(f"   RMSE: {poly_train_rmse:.4f}")
print(f"   MAE:  {poly_train_mae:.4f}")
print(f"   R²:   {poly_train_r2:.4f}")

print(f"\n📊 Testing Set:")
print(f"   MSE:  {poly_test_mse:.4f}")
print(f"   RMSE: {poly_test_rmse:.4f}")
print(f"   MAE:  {poly_test_mae:.4f}")
print(f"   R²:   {poly_test_r2:.4f}")
print("="*70)

## 7. Model 3: Decision Tree Regressor

**Theory:** Learns decision rules from data features to make predictions

**Method:** Recursively splits data based on features to minimize variance

**Advantages:** Can capture non-linear relationships without feature engineering

In [ ]:
print("Training Decision Tree Regressor Model...")

dt_model = DecisionTreeRegressor(
    max_depth=15,
    min_samples_split=20,
    min_samples_leaf=10,
    random_state=42
)
dt_model.fit(X_train, y_train)

# Predictions
y_train_pred_dt = dt_model.predict(X_train)
y_test_pred_dt = dt_model.predict(X_test)

# Evaluate
dt_train_mse = mean_squared_error(y_train, y_train_pred_dt)
dt_test_mse = mean_squared_error(y_test, y_test_pred_dt)
dt_train_rmse = np.sqrt(dt_train_mse)
dt_test_rmse = np.sqrt(dt_test_mse)
dt_train_mae = mean_absolute_error(y_train, y_train_pred_dt)
dt_test_mae = mean_absolute_error(y_test, y_test_pred_dt)
dt_train_r2 = r2_score(y_train, y_train_pred_dt)
dt_test_r2 = r2_score(y_test, y_test_pred_dt)

print("\n" + "="*70)
print("DECISION TREE REGRESSOR RESULTS")
print("="*70)
print(f"\n📊 Training Set:")
print(f"   MSE:  {dt_train_mse:.4f}")
print(f"   RMSE: {dt_train_rmse:.4f}")
print(f"   MAE:  {dt_train_mae:.4f}")
print(f"   R²:   {dt_train_r2:.4f}")

print(f"\n📊 Testing Set:")
print(f"   MSE:  {dt_test_mse:.4f}")
print(f"   RMSE: {dt_test_rmse:.4f}")
print(f"   MAE:  {dt_test_mae:.4f}")
print(f"   R²:   {dt_test_r2:.4f}")
print("="*70)

## 8. Model 4: Random Forest Regressor

**Theory:** Ensemble of decision trees - combines multiple trees to improve predictions

**Method:** 
- Bootstrap aggregating (bagging) of decision trees
- Random feature selection at each split
- Final prediction = average of all tree predictions

**Advantages:** More robust and accurate than single decision tree

In [ ]:
print("Training Random Forest Regressor Model...")

rf_model = RandomForestRegressor(
    n_estimators=100,
    max_depth=20,
    min_samples_split=10,
    min_samples_leaf=4,
    random_state=42,
    n_jobs=-1,
    verbose=1
)
rf_model.fit(X_train, y_train)

# Predictions
y_train_pred_rf = rf_model.predict(X_train)
y_test_pred_rf = rf_model.predict(X_test)

# Evaluate
rf_train_mse = mean_squared_error(y_train, y_train_pred_rf)
rf_test_mse = mean_squared_error(y_test, y_test_pred_rf)
rf_train_rmse = np.sqrt(rf_train_mse)
rf_test_rmse = np.sqrt(rf_test_mse)
rf_train_mae = mean_absolute_error(y_train, y_train_pred_rf)
rf_test_mae = mean_absolute_error(y_test, y_test_pred_rf)
rf_train_r2 = r2_score(y_train, y_train_pred_rf)
rf_test_r2 = r2_score(y_test, y_test_pred_rf)

print("\n" + "="*70)
print("RANDOM FOREST REGRESSOR RESULTS")
print("="*70)
print(f"\n📊 Training Set:")
print(f"   MSE:  {rf_train_mse:.4f}")
print(f"   RMSE: {rf_train_rmse:.4f}")
print(f"   MAE:  {rf_train_mae:.4f}")
print(f"   R²:   {rf_train_r2:.4f}")

print(f"\n📊 Testing Set:")
print(f"   MSE:  {rf_test_mse:.4f}")
print(f"   RMSE: {rf_test_rmse:.4f}")
print(f"   MAE:  {rf_test_mae:.4f}")
print(f"   R²:   {rf_test_r2:.4f}")
print("="*70)

## 9. Model 5: Neural Network (PyTorch)

**Theory:** Multi-layer perceptron with backpropagation

**Architecture:**
- Input layer: Number of features
- Hidden layer 1: 64 neurons + ReLU activation
- Hidden layer 2: 32 neurons + ReLU activation  
- Hidden layer 3: 16 neurons + ReLU activation
- Output layer: 1 neuron (regression output)

**Training:** Adam optimizer, MSE loss, backpropagation

### 9.1 Prepare Data for PyTorch

In [ ]:
# Convert to PyTorch tensors
X_train_tensor = torch.FloatTensor(X_train.values).to(device)
y_train_tensor = torch.FloatTensor(y_train.values).reshape(-1, 1).to(device)
X_test_tensor = torch.FloatTensor(X_test.values).to(device)
y_test_tensor = torch.FloatTensor(y_test.values).reshape(-1, 1).to(device)

# Create DataLoader for batch training
batch_size = 64
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

print("✅ Data converted to PyTorch tensors")
print(f"\n📦 Batch size: {batch_size}")
print(f"📦 Number of batches: {len(train_loader)}")
print(f"\n🔢 Tensor shapes:")
print(f"   X_train: {X_train_tensor.shape}")
print(f"   y_train: {y_train_tensor.shape}")
print(f"   X_test: {X_test_tensor.shape}")
print(f"   y_test: {y_test_tensor.shape}")

### 9.2 Define Neural Network Architecture

In [ ]:
class SolarPowerNN(nn.Module):
    def __init__(self, input_size):
        super(SolarPowerNN, self).__init__()
        
        # Define layers
        self.fc1 = nn.Linear(input_size, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 16)
        self.fc4 = nn.Linear(16, 1)
        
        # Activation function
        self.relu = nn.ReLU()
        
        # Dropout for regularization
        self.dropout = nn.Dropout(0.2)
    
    def forward(self, x):
        # Forward pass with ReLU activations
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.relu(self.fc3(x))
        x = self.dropout(x)
        x = self.fc4(x)  # No activation for regression output
        return x

# Initialize model
input_size = X_train.shape[1]
nn_model = SolarPowerNN(input_size).to(device)

print("\n" + "="*70)
print("NEURAL NETWORK ARCHITECTURE")
print("="*70)
print(nn_model)
print("\n" + "="*70)

# Count parameters
total_params = sum(p.numel() for p in nn_model.parameters())
trainable_params = sum(p.numel() for p in nn_model.parameters() if p.requires_grad)
print(f"\nTotal parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")

### 9.3 Train Neural Network

In [ ]:
# Loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(nn_model.parameters(), lr=0.001)

# Training parameters
num_epochs = 100
train_losses = []
val_losses = []

print("\n" + "="*70)
print("TRAINING NEURAL NETWORK")
print("="*70)
print(f"Epochs: {num_epochs}")
print(f"Batch size: {batch_size}")
print(f"Learning rate: 0.001")
print(f"Optimizer: Adam")
print(f"Loss function: MSE")
print("\nStarting training...\n")

# Training loop
for epoch in range(num_epochs):
    nn_model.train()
    epoch_loss = 0.0
    
    for batch_X, batch_y in train_loader:
        # Forward pass
        outputs = nn_model(batch_X)
        loss = criterion(outputs, batch_y)
        
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
    
    # Average loss for the epoch
    avg_train_loss = epoch_loss / len(train_loader)
    train_losses.append(avg_train_loss)
    
    # Validation loss
    nn_model.eval()
    with torch.no_grad():
        val_outputs = nn_model(X_test_tensor)
        val_loss = criterion(val_outputs, y_test_tensor).item()
        val_losses.append(val_loss)
    
    # Print progress every 10 epochs
    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}] - Train Loss: {avg_train_loss:.4f}, Val Loss: {val_loss:.4f}")

print("\n✅ Training completed!")

### 9.4 Evaluate Neural Network

In [ ]:
# Make predictions
nn_model.eval()
with torch.no_grad():
    y_train_pred_nn_tensor = nn_model(X_train_tensor)
    y_test_pred_nn_tensor = nn_model(X_test_tensor)

# Convert to numpy
y_train_pred_nn = y_train_pred_nn_tensor.cpu().numpy().flatten()
y_test_pred_nn = y_test_pred_nn_tensor.cpu().numpy().flatten()

# Evaluate
nn_train_mse = mean_squared_error(y_train, y_train_pred_nn)
nn_test_mse = mean_squared_error(y_test, y_test_pred_nn)
nn_train_rmse = np.sqrt(nn_train_mse)
nn_test_rmse = np.sqrt(nn_test_mse)
nn_train_mae = mean_absolute_error(y_train, y_train_pred_nn)
nn_test_mae = mean_absolute_error(y_test, y_test_pred_nn)
nn_train_r2 = r2_score(y_train, y_train_pred_nn)
nn_test_r2 = r2_score(y_test, y_test_pred_nn)

print("\n" + "="*70)
print("NEURAL NETWORK RESULTS")
print("="*70)
print(f"\n📊 Training Set:")
print(f"   MSE:  {nn_train_mse:.4f}")
print(f"   RMSE: {nn_train_rmse:.4f}")
print(f"   MAE:  {nn_train_mae:.4f}")
print(f"   R²:   {nn_train_r2:.4f}")

print(f"\n📊 Testing Set:")
print(f"   MSE:  {nn_test_mse:.4f}")
print(f"   RMSE: {nn_test_rmse:.4f}")
print(f"   MAE:  {nn_test_mae:.4f}")
print(f"   R²:   {nn_test_r2:.4f}")
print("="*70)

### 9.5 Plot Training History

In [ ]:
# Plot training history
plt.figure(figsize=(12, 5))
plt.plot(train_losses, label='Training Loss', linewidth=2)
plt.plot(val_losses, label='Validation Loss', linewidth=2)
plt.title('Neural Network Training History', fontsize=14, fontweight='bold')
plt.xlabel('Epoch', fontsize=12)
plt.ylabel('Loss (MSE)', fontsize=12)
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('nn_training_history.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n✅ Training history plot saved as 'nn_training_history.png'")

## 10. Model Comparison

In [ ]:
# Create comparison dataframe
results = pd.DataFrame({
    'Model': ['Linear Regression', 'Polynomial Regression', 'Decision Tree', 'Random Forest', 'Neural Network'],
    'Train_RMSE': [lr_train_rmse, poly_train_rmse, dt_train_rmse, rf_train_rmse, nn_train_rmse],
    'Test_RMSE': [lr_test_rmse, poly_test_rmse, dt_test_rmse, rf_test_rmse, nn_test_rmse],
    'Train_MAE': [lr_train_mae, poly_train_mae, dt_train_mae, rf_train_mae, nn_train_mae],
    'Test_MAE': [lr_test_mae, poly_test_mae, dt_test_mae, rf_test_mae, nn_test_mae],
    'Train_R²': [lr_train_r2, poly_train_r2, dt_train_r2, rf_train_r2, nn_train_r2],
    'Test_R²': [lr_test_r2, poly_test_r2, dt_test_r2, rf_test_r2, nn_test_r2]
})

# Sort by Test R² (descending)
results = results.sort_values('Test_R²', ascending=False).reset_index(drop=True)

print("\n" + "="*100)
print("MODEL COMPARISON RESULTS - SOLAR POWER PREDICTION")
print("="*100)
print(results.to_string(index=False))
print("="*100)

# Find best model
best_model = results.iloc[0]['Model']
best_r2 = results.iloc[0]['Test_R²']
best_rmse = results.iloc[0]['Test_RMSE']
best_mae = results.iloc[0]['Test_MAE']

print(f"\n🏆 Best Model: {best_model}")
print(f"   Test R² Score: {best_r2:.4f}")
print(f"   Test RMSE: {best_rmse:.4f}")
print(f"   Test MAE: {best_mae:.4f}")

### 10.1 Visualize Model Comparison

In [ ]:
# Create comparison plots
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# RMSE comparison
x_pos = np.arange(len(results))
axes[0].bar(x_pos - 0.2, results['Train_RMSE'], 0.4, label='Train', color='steelblue', alpha=0.8)
axes[0].bar(x_pos + 0.2, results['Test_RMSE'], 0.4, label='Test', color='coral', alpha=0.8)
axes[0].set_xticks(x_pos)
axes[0].set_xticklabels(results['Model'], rotation=45, ha='right')
axes[0].set_ylabel('RMSE (Lower is Better)', fontweight='bold')
axes[0].set_title('Root Mean Squared Error', fontsize=14, fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3, axis='y')

# MAE comparison
axes[1].bar(x_pos - 0.2, results['Train_MAE'], 0.4, label='Train', color='steelblue', alpha=0.8)
axes[1].bar(x_pos + 0.2, results['Test_MAE'], 0.4, label='Test', color='coral', alpha=0.8)
axes[1].set_xticks(x_pos)
axes[1].set_xticklabels(results['Model'], rotation=45, ha='right')
axes[1].set_ylabel('MAE (Lower is Better)', fontweight='bold')
axes[1].set_title('Mean Absolute Error', fontsize=14, fontweight='bold')
axes[1].legend()
axes[1].grid(True, alpha=0.3, axis='y')

# R² comparison
axes[2].bar(x_pos - 0.2, results['Train_R²'], 0.4, label='Train', color='steelblue', alpha=0.8)
axes[2].bar(x_pos + 0.2, results['Test_R²'], 0.4, label='Test', color='green', alpha=0.8)
axes[2].set_xticks(x_pos)
axes[2].set_xticklabels(results['Model'], rotation=45, ha='right')
axes[2].set_ylabel('R² Score (Higher is Better)', fontweight='bold')
axes[2].set_title('R² Score', fontsize=14, fontweight='bold')
axes[2].legend()
axes[2].grid(True, alpha=0.3, axis='y')

plt.suptitle('Solar Power Prediction - Model Comparison', fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig('model_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n✅ Comparison plot saved as 'model_comparison.png'")

## 11. Prediction Visualization

In [ ]:
# Plot predictions vs actual for all models
fig, axes = plt.subplots(3, 2, figsize=(16, 18))
axes = axes.flatten()

# Sample indices for visualization (first 500 test points)
sample_size = min(500, len(y_test))

models_data = [
    ('Linear Regression', y_test_pred_lr, lr_test_r2),
    ('Polynomial Regression', y_test_pred_poly, poly_test_r2),
    ('Decision Tree', y_test_pred_dt, dt_test_r2),
    ('Random Forest', y_test_pred_rf, rf_test_r2),
    ('Neural Network', y_test_pred_nn, nn_test_r2)
]

for idx, (name, pred, r2) in enumerate(models_data):
    axes[idx].plot(y_test.values[:sample_size], label='Actual', linewidth=2, alpha=0.7)
    axes[idx].plot(pred[:sample_size], label='Predicted', linewidth=2, alpha=0.7)
    axes[idx].set_title(f'{name} (R²={r2:.4f})', fontsize=12, fontweight='bold')
    axes[idx].set_xlabel('Sample Index')
    axes[idx].set_ylabel('DC Power')
    axes[idx].legend()
    axes[idx].grid(True, alpha=0.3)

# Hide last subplot
axes[5].axis('off')

plt.suptitle('Solar Power Prediction - All Models Comparison', fontsize=16, fontweight='bold', y=1.00)
plt.tight_layout()
plt.savefig('predictions_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n✅ Predictions plot saved as 'predictions_comparison.png'")

## 12. Feature Importance (Random Forest)

In [ ]:
# Feature importance from Random Forest
feature_importance = pd.DataFrame({
    'Feature': features,
    'Importance': rf_model.feature_importances_
}).sort_values('Importance', ascending=False)

# Plot top 15 features
plt.figure(figsize=(12, 8))
top_n = min(15, len(feature_importance))
plt.barh(feature_importance['Feature'][:top_n], feature_importance['Importance'][:top_n], color='steelblue')
plt.xlabel('Importance', fontweight='bold', fontsize=12)
plt.ylabel('Feature', fontweight='bold', fontsize=12)
plt.title(f'Top {top_n} Feature Importance (Random Forest)', fontsize=14, fontweight='bold')
plt.gca().invert_yaxis()
plt.grid(True, alpha=0.3, axis='x')
plt.tight_layout()
plt.savefig('feature_importance.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n✅ Feature importance plot saved as 'feature_importance.png'")
print(f"\nTop 10 Most Important Features:")
print(feature_importance.head(10).to_string(index=False))

## 13. Summary and Conclusions

In [ ]:
print("\n" + "="*80)
print("SOLAR POWER PREDICTION - FINAL SUMMARY")
print("="*80)

print("\n📊 Models Trained:")
for idx, row in results.iterrows():
    print(f"  {idx+1}. {row['Model']}:")
    print(f"     Test R² = {row['Test_R²']:.4f}, RMSE = {row['Test_RMSE']:.4f}, MAE = {row['Test_MAE']:.4f}")

print(f"\n🏆 Best Performing Model: {best_model}")
print(f"   Test R² Score: {best_r2:.4f}")
print(f"   Test RMSE: {best_rmse:.4f}")
print(f"   Test MAE: {best_mae:.4f}")

print("\n📁 Generated Outputs:")
print("  - model_comparison.png")
print("  - predictions_comparison.png")
print("  - feature_importance.png")
print("  - nn_training_history.png")

print("\n✅ All models successfully trained and evaluated!")
print("="*80)